In [1]:
import copy
import numpy as np
import laspy as lp
import os
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def las_to_open3d(las_source_path, target_path):
    las = lp.read(las_source_path)
    points = np.vstack((las.x, las.y, las.z)).transpose()

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    # Optional: add colors if available
    if hasattr(las, "red") and hasattr(las, "green") and hasattr(las, "blue"):
        colors = np.vstack((las.red, las.green, las.blue)).transpose()
        colors = colors / 65535.0 if np.max(colors) > 1.0 else colors  # Normalize if needed
        pcd.colors = o3d.utility.Vector3dVector(colors)

    # Visualize or save
    #o3d.visualization.draw_geometries([pcd])
    o3d.io.write_point_cloud(target_path, pcd)

In [3]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
                                      #zoom=0.4459,
                                      #front=[0.9288, -0.2951, -0.2242],
                                      #lookat=[1.6784, 2.0612, 1.4451],
                                      #up=[-0.3402, -0.9189, -0.1996])

In [4]:
source_path = '/home/diego/Downloads/250329_153004_100m_10ms_100khz_plena-luz/clasificadas_sin_talud/km-11-sector-b_100m_10ms_100khz_plena-luz_0_0.las'
target_path = '/home/diego/Downloads/250329_105110_100m_10ms_100khz_madrugada/clasificadas_sin_talud/km-11-sector-b_100m_10ms_100khz_madrugada_0_0.las'

In [5]:
pcd_source_path = source_path.replace('.las', '.pcd')
pcd_target_path = target_path.replace('.las', '.pcd')
# Check if the pcd_source_path and pcd_target_path already exist
if not os.path.exists(pcd_source_path) or not os.path.exists(pcd_target_path):
    # Convert LAS to Open3D PointCloud
    las_to_open3d(source_path, pcd_source_path)
    las_to_open3d(target_path, pcd_target_path)

In [6]:
source = o3d.io.read_point_cloud(pcd_source_path)
target = o3d.io.read_point_cloud(pcd_target_path)

In [7]:
source.estimate_normals()
target.estimate_normals()

In [9]:
threshold = 0.02
trans_init = np.asarray([[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0],
                             [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
#draw_registration_result(source, target, trans_init)

In [10]:
print("Apply point-to-plane ICP")
reg_p2l = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPlane())
print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
#draw_registration_result(source, target, reg_p2l.transformation)

Apply point-to-plane ICP
RegistrationResult with fitness=6.805482e-01, inlier_rmse=7.647170e-03, and correspondence_set size of 3329929
Access transformation to get result.
Transformation is:
[[ 9.99999991e-01 -1.40893015e-08  1.36041376e-04 -2.03413129e-01]
 [ 5.37689522e-10  9.99999995e-01  9.96138989e-05 -1.92749128e-01]
 [-1.36041377e-04 -9.96138979e-05  9.99999986e-01  8.14256745e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [11]:
import pyvista as pv
import laspy as lp

In [12]:
transformation = reg_p2l.transformation
print(f"Transformation is: {transformation}")

Transformation is: [[ 9.99999991e-01 -1.40893015e-08  1.36041376e-04 -2.03413129e-01]
 [ 5.37689522e-10  9.99999995e-01  9.96138989e-05 -1.92749128e-01]
 [-1.36041377e-04 -9.96138979e-05  9.99999986e-01  8.14256745e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [ ]:
# transformed_source = copy.deepcopy(source)
# transformed_source.transform(transformation)
# points = np.asarray(transformed_source.points)

In [13]:
import laspy
import numpy as np

# Read existing LAS
las = laspy.read(source_path)

src_las_data_points_transformed = np.dot(
     transformation[:3, :3], las.xyz.T).T + transformation[:3, 3]

# Transform the point cloud using the transformation matrix
#src_las_data_points_transformed = points

# Overwrite coordinates with your transformed point cloud
las.x = src_las_data_points_transformed[:, 0]
las.y = src_las_data_points_transformed[:, 1]
las.z = src_las_data_points_transformed[:, 2]

# Save as a new file (don’t overwrite original unless sure)
output_path = source_path.replace('.las', '_transformed.las')
print(f"Saving transformed LAS to {output_path}")
las.write(output_path)

Saving transformed LAS to /home/diego/Downloads/250329_153004_100m_10ms_100khz_plena-luz/clasificadas_sin_talud/km-11-sector-b_100m_10ms_100khz_plena-luz_0_0_transformed.las
